In [1]:
#%%capture

# vincemauro - Da lanciare sempre per verificare che siano soddisfatti i requisiti tecnici
# capture disabilita l'output di pip. Rimuoverlo per leggere eventuali errori.

#%pip install -r ./requirements.txt

import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import ipywidgets as widgets        

In [2]:
#vincemauro - Da lanciare sempre per caricare i dati storici

#import del file, questo blocco è troppo rigido, va automatizzato nella class in modo da utilizzare tutta la serie storica disponibile degli strumenti che si vogliono in portafoglio (cioè con peso maggiore di 0)
#imported_dataframe= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TBTFolio - testing/Timeseries.csv", sep = ";")
imported_dataframe= pd.read_csv("./Timeseries.csv", sep = ";")
# Converte la colonna Date in datetime
imported_dataframe["Date"] = pd.to_datetime(imported_dataframe["Date"], dayfirst=True, errors="coerce")
# Tieni solo le righe con Date >= cutoff
imported_dataframe = imported_dataframe[(imported_dataframe["Date"]>=pd.to_datetime("2000-01-03"))].copy()
imported_dataframe = imported_dataframe[(imported_dataframe["Date"]<=pd.to_datetime("2025-09-04"))].copy().reset_index(drop=True)
#print(imported_dataframe)

In [3]:
#vincemauro - Da lanciare sempre per importare la classe portfolio_evo e definire la funzione di backtest

%reload_ext autoreload
%autoreload 2

from classes import portfolio_evo

def backtest(initial_balance, start_date, end_date, initial_w):
    ## RUN PRINCIPALE ##
    # Allocazione su indici di portafoglio:
    # Index 1  # Index 2 # Index 3 # Index 4 ...
    # VTSIM SPYSIM  VXUSSIM GLDSIM  CASHX SHYSIM  IEFSIM  TLTSIM  ZROZSIM KMLMSIM DBMFSIM

    if np.sum(initial_w) != 1:
        print("L'allocazione non somma a 1")
        sys.exit()

    ptf = portfolio_evo(initial_balance = initial_balance,
                    transac_cost_rate= 0.0029, #0.19% commissioni + 0.1% spread
                    exp_rate=0.002,            #0.2% imposta di bollo + 0% tracking error
                    tax_rate = 0.26,           #26% conservativo
                    rebalance_threshold = 0.1,
                    initial_w = initial_w,
                    imported_dataframe= imported_dataframe,
                    start_date = start_date,
                    end_date = end_date,
                    stock_price_normalization= True)

    df_log       = pd.DataFrame(index = ptf.date)
    df_log_delta = pd.DataFrame(index = ptf.date)

    for i in ptf.StockPrice.index:
        ptf.reset_tax_transaccost()
        ptf.reset_delta_notional()
        
        StockPrice = ptf.StockPrice.loc[i,:]
        ptf.update_AssetValue_weight(StockPrice)
        
        if ptf.check_rebalance():
            #print(i)
            #print(f"check_rebalance {ptf.check_rebalance()}")
            #print(f"StockPrice\n {StockPrice}")
            ptf.update_notional_tax_transaccost(StockPrice)

        ptf.update_Return(StockPrice)    
        ptf.update_TotValue(StockPrice)    
        ptf.update_NetTotValue(StockPrice)

        df_log.loc[ptf.date[i], "Return"]                = ptf.PercReturn
        df_log.loc[ptf.date[i], "Compound Return"]       = ptf.CompoundReturn
        df_log.loc[ptf.date[i], "TotValue"]              = ptf.TotValue
        df_log.loc[ptf.date[i], "Taxes"]                 = ptf.tax
        df_log.loc[ptf.date[i], "TransacCost"]           = ptf.TransactionalCost
        df_log.loc[ptf.date[i], ptf.IndexName]           = ptf.AssetValue
        df_log_delta.loc[ptf.date[i], ptf.IndexName]     = ptf.delta_notional*StockPrice

    df_log.to_excel("output_ptf.xlsx")
    df_log_delta.to_excel("output_ptf_delta.xlsx")

    print(f"Orizzonte temporale   {min(ptf.date).date()} / {max(ptf.date).date()}")
    print(f"Anni in simulazione   {round(((max(ptf.date) - min(ptf.date)).days / 365.25), 2)}")
    print(f"CAGR                  {(ptf.CompoundReturn**(1/(round(((max(ptf.date) - min(ptf.date)).days / 365.25), 2))) -1) * 100:.2f}%")
    print(f"Total compound return {ptf.CompoundReturn * 100:.2f}%")
    print(f"Capitale iniziale {ptf.StartValue}") #20260131 vincemauro
    print(f"Capitale finale {ptf.TotValue:.2f}") #20260131 vincemauro
    plt.semilogy(df_log.index,df_log['Compound Return'])
    plt.xlabel('Data')
    plt.ylabel('Compound Return %')
    #plt.title('Grafico')
    #plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [7]:
#vincemauro - Da lanciare per eseguire un backtest con GUI

from IPython.display import display

# Creazione dei controlli
ticker_VT   = widgets.FloatSlider(value=0.6, min=0, max=1.0, step=0.01, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f', description='VT:')
ticker_SPY  = widgets.FloatSlider(value=0, min=0, max=1.0, step=0.01, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f', description='SPY:')
ticker_VXUS = widgets.FloatSlider(value=0, min=0, max=1.0, step=0.01, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f', description='VXUS:')
ticker_GLD  = widgets.FloatSlider(value=0.1, min=0, max=1.0, step=0.01, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f', description='GLD:')
ticker_CASH = widgets.FloatSlider(value=0, min=0, max=1.0, step=0.01, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f', description='CASH:')
ticker_SHY  = widgets.FloatSlider(value=0, min=0, max=1.0, step=0.01, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f', description='SHY:')
ticker_IEF  = widgets.FloatSlider(value=0, min=0, max=1.0, step=0.01, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f', description='IEF:')
ticker_TLT  = widgets.FloatSlider(value=0, min=0, max=1.0, step=0.01, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f', description='TLT:')
ticker_ZROZ = widgets.FloatSlider(value=0.25, min=0, max=1.0, step=0.01, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f', description='ZROZ:')
ticker_KMLM = widgets.FloatSlider(value=0, min=0, max=1.0, step=0.01, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f', description='KMLM:')
ticker_DBMF = widgets.FloatSlider(value=0.05, min=0, max=1.0, step=0.01, disabled=False, continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f', description='DBMF:')

capitale_input = widgets.IntSlider(value=10000, min=1000, max=100000, step=1000, description='Capitale:')

data_inizio = widgets.DatePicker(value=pd.to_datetime("2025-01-01"), description='Data iniziale', disabled=False)
data_fine = widgets.DatePicker(value=pd.to_datetime("2025-09-01"), description='Data finale', disabled=False)

# Finestra di output
output = widgets.Output()

# Bottone per avviare il backtest
button = widgets.Button(description="Esegui Backtest")

ticker_box = widgets.VBox( #colonna sinistra con i ticker
    [
        ticker_VT,
        ticker_SPY,
        ticker_VXUS,
        ticker_GLD,
        ticker_CASH,
        ticker_SHY,
        ticker_IEF,
        ticker_TLT,
        ticker_ZROZ,
        ticker_KMLM,
        ticker_DBMF
    ]
)

center_box = widgets.VBox( #colonna centrale con capitale, date e pulsante
    [
        capitale_input,
        data_inizio,
        data_fine,
        button
    ]
)

all_w = widgets.HBox(
    [
        ticker_box,
        center_box,
        output
    ]
)

display(all_w)

def on_button_clicked(b):
    with output:
        output.clear_output()
        w = [
            ticker_VT.value,
            ticker_SPY.value,
            ticker_VXUS.value,
            ticker_GLD.value,
            ticker_CASH.value,
            ticker_SHY.value,
            ticker_IEF.value,
            ticker_TLT.value,
            ticker_ZROZ.value,
            ticker_KMLM.value,
            ticker_DBMF.value
            ]
        backtest(capitale_input.value, data_inizio.value, data_fine.value, w)

button.on_click(on_button_clicked)
display(button)

Button(description='Esegui Backtest', style=ButtonStyle())